Deriving z.

In [ ]:
# modifying "reading in financial data" to 32 assets and right T value

import numpy as np
import pandas as pd

# You may choose to select different parameters/values
number_assets = 32
T  = 392
# Read returns
df = pd.read_csv('returns_data.txt',delim_whitespace=True)

Rraw = df.values.T

# Select the first N,T assets and scenarios, you may choose a different strategy if you would like to do so.
R = Rraw[:number_assets,:T]

# Expected return of each asset
expected_returns = np.mean(R, axis = 1)

# Covariance matrix of asset returns
covariance_matrix = np.cov(R)

In [ ]:
# import random weight function from random_weights.ipynb

def generate_random_weights(number_of_assets):
    weights = np.random.rand(number_of_assets) # Generate random numbers
    weights /= np.sum(weights) # Normalize so that the sum is 1
    return weights

# Test the function with 32 assets
n_assets = 32
random_weights = generate_random_weights(n_assets)
random_weights, sum(random_weights)


In [ ]:
# Redefining the calculate_historical_VaR function
def calculate_historical_VaR(weights, mu_R, confidence_level=0.95):
    # Calculate portfolio returns for the given weights
    portfolio_returns = np.dot(mu_R, np.transpose(np.array(weights)))
    # Calculate the VaR at the specified confidence level
    VaR = np.percentile(portfolio_returns, 100 * (1 - confidence_level))
    return -VaR

In [ ]:
# Define a function to simulate the random generation of weights and calculation of VaR
def simulate_VaR(num_simulations, num_assets, mu_R, confidence_level=0.95):
    VaR_results = []
    weights_list = []

    for _ in range(num_simulations):
        weights = generate_random_weights(num_assets)
        VaR = calculate_historical_VaR(weights, mu_R, confidence_level)
        VaR_results.append(VaR)
        weights_list.append(weights)
    
    return VaR_results, weights_list

In [ ]:
# Example usage
num_simulations = 1000  # The number of times we want to run the simulation
num_assets = 32  # Number of assets in the portfolio

# We need the mu_R matrix to run the simulations which should be the historical returns matrix
# For the sake of the example, let's create a dummy mu_R with random values
# Normally, you would use actual historical returns data for your assets here
mu_R = np.random.randn(1000, num_assets)  # 1000 scenarios, 32 assets

# Run the simulation
VaR_results, weights_list = simulate_VaR(num_simulations, num_assets, mu_R)

# Display the first few results to ensure it's working
VaR_results[:5], weights_list[:5]

In [ ]:
# let's now actually calculate z

# Parameters
number_of_assets = 32
number_of_portfolios = 1000  # The number of random portfolios you want to generate
confidence_level = 0.95

# Assuming mu_R is the matrix of historical returns for the assets
# Replace this with your actual historical returns data
mu_R = expected_returns  # This should be the historical returns data for your assets

# Generate the random weights for the portfolios and calculate the VaR for each
VaRs = []
for _ in range(number_of_portfolios):
    weights = generate_random_weights(number_of_assets)
    VaR = calculate_historical_VaR(weights, mu_R, confidence_level)
    VaRs.append(VaR)

# Calculate z
mean_VaR = np.mean(VaRs)
median_VaR = np.median(VaRs)
z = mean_VaR  # or median_VaR, depending on your specific requirements

print(f"The mean VaR at a {confidence_level*100}% confidence level is: {mean_VaR}")
print(f"The median VaR at a {confidence_level*100}% confidence level is: {median_VaR}")
